# HW01 PLA
> 1041621 蘇建文

## The roadmap of code
![](https://raw.githubusercontent.com/johnny12150/DSS_P2P/f3eabbf965fea515e87094703e00d9c34b3bac6b/notebook/img/Slide10.jpg)

### 01. Dataset
> 資料來源: https://www.kaggle.com/c/titanic/data

檢視資料

In [60]:
import numpy as np
import pandas as pd

DF = pd.read_csv('train.csv')
DF.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 02. 資料前處理

#### 選取feature & label

In [61]:
train_feature = DF.drop(['Name', 'Ticket', 'PassengerId','Survived','Cabin'], axis=1)
train_labels = DF['Survived'].copy()

#### 處理空值、異常值 
檢查空值

In [4]:
train_feature.isnull().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

將年紀的空值用平均值補

In [62]:
median_age = train_feature['Age'].dropna().median()
# 取出Age是Nan的row後，將該row的Age以平均數取代之
train_feature.loc[ (train_feature['Age'].isnull()), 'Age'] = median_age

用眾數補embarked

In [63]:
train_feature['Embarked'] = train_feature['Embarked'].fillna(train_feature['Embarked'].mode()[0])

#### 製作新feature
判斷Name裡的稱謂

In [22]:
DF['Title'] = DF['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())

減少稱謂的類別

In [23]:
title_dict = {
                "Mr" :        "Mr",
                "Miss" :      "Miss",
                "Mrs" :       "Mrs",
                "Master" :    "Master",
                "Dr":         "Scholar",
                "Rev":        "Religious",
                "Col":        "Officer",
                "Major":      "Officer",
                "Mlle":       "Miss",
                "Don":        "Noble",
                "the Countess":"Noble",
                "Ms":         "Mrs",
                "Mme":        "Mrs",
                "Capt":       "Noble",
                "Lady" :      "Noble",
                "Sir" :       "Noble",
                "Jonkheer":   "Noble"
            }

DF['TitleGroup'] = DF['Title'].map(title_dict)

### 03. One Hot Encoding
將Embarked mapping成數值

In [64]:
train_feature['Embarked'] = train_feature['Embarked'].map({'C': 0, 'Q': 1, 'S':2}).astype(int)

將TitleGroup mapping成數值

In [24]:
# train_feature = pd.concat([train_feature, pd.get_dummies(DF['TitleGroup'])], axis=1)

In [ ]:
train_feature['TitleGroup'] = train_feature['TitleGroup'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, }).astype(int)

將sex mapping成數值

In [68]:
train_feature['Sex'] = train_feature['Sex'].map({'male': 0, 'female': 1}).astype(int)

### 04. Scaler
將數值的feature做正規化

In [39]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_feature[['Pclass', 'Age', 'Parch', 'Fare']] = scaler.fit_transform(train_feature[['Pclass', 'Age', 'Parch', 'Fare']])
train_feature.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.827377,0,-0.565736,1,-0.473674,-0.502445,2
1,-1.566107,1,0.663861,1,-0.473674,0.786845,0
2,0.827377,1,-0.258337,0,-0.473674,-0.488854,2
3,-1.566107,1,0.433312,1,-0.473674,0.420730,2
4,0.827377,0,0.433312,0,-0.473674,-0.486337,2


### 05. 演算法

### 1. PLA演算法

參數設定

In [65]:
weights = np.zeros(len(train_feature.columns))

learningRate = 0.01

Iterations = 15
# 紀錄error變化
errors = []

激勵函數

In [ ]:
def sign(z):
    if z > 0:
        return 1
    else:
        return -1

簡單版

In [66]:
def pla_simple(data, labels, iterations):
    for numIter in range(Iterations):
        # print("iteration {}".format(numIter))
        err = 0  # reset the error counter
        # For each handwritten digit in training set,
        for ix in range(len(data.index)):
            # select a row as input vector 
            inputVector = data.iloc[ix]
            # uses global variable in the function
            global weights
            # take the dot product of input and weight
            dotproduct = np.dot(inputVector, weights)
            # 經過激勵函數
            estimatedLabel = np.sign(dotproduct)
            
            # 如果sign回傳是0, 看作小於0
            if estimatedLabel == 0:
                estimatedLabel = -1
                
            actualLabel = labels[ix];
            # PLA的二元分類, label需為1和-1, 故將原本是0的轉為-1
            if actualLabel == 0:
                actualLabel = -1
                
            # 比較預測的label與答案
            if actualLabel != estimatedLabel:   
                weights = weights + actualLabel*learningRate*inputVector
                err+=1
                
        global errors
        errors.append( (err*1.0) / len(data) )  

In [69]:
pla_simple(train_feature, train_labels, Iterations)
errors[14]

0.3254769921436588

訓練後的權重

In [42]:
weights

Pclass      0.050000
Age         1.900000
SibSp       0.050000
Parch       0.000000
Fare        3.564165
Embarked    0.000000
dtype: float64

### with sgd

In [52]:
weights = np.zeros(len(train_feature.columns))

learningRate = 0.1

bias = 0

Iterations = 20
# 紀錄error變化
errors = []

In [53]:
def pla_sgd(data, labels, iterations):
    for numIter in range(Iterations):
        err = 0  # reset the error counter
        # For each handwritten digit in training set,
        for ix in range(len(data.index)):
            # select a row as input vector 
            inputVector = data.iloc[ix]
            # uses global variable in the function
            global weights
            # take the dot product of input and weight
            dotproduct = np.dot(inputVector, weights)
            # 經過激勵函數
            estimatedLabel = np.sign(dotproduct)
            
            # 如果sign回傳是0, 看作小於0
            if estimatedLabel == 0:
                estimatedLabel = -1
            actualLabel = labels[ix];
            # 比較預測的label與答案
            if actualLabel != estimatedLabel:
                # SGD: expect - predict 計算 loss
                weights = weights + (actualLabel - estimatedLabel)*inputVector*learningRate
                # 訓練bias
                global bias
                bias = bias + (actualLabel - estimatedLabel)*learningRate
                err+=1
                
        global errors
        errors.append( (err*1.0) / len(data) )  

In [54]:
pla_sgd(train_feature, train_labels, Iterations)
errors[19]

0.6947250280583613

到特定iteration停下

In [ ]:
def pla(data):
    w = np.array([0.,0.,0.])
    error = 1
    iterator = 0
    while error != 0:
        error = 0
        for i in range(len(data)):
            x, y = np.concatenate((np.array([1.]), np.array(data.iloc[i])[:2])), np.array(data.iloc[i])[2]
            if sign(np.dot(w,x)) != y:
                print("iterator: "+str(iterator))
                iterator += 1
                error += 1
                

                # 前一個 Decision boundary 的法向量
                if w[1] != 0:
                    x_last_decision_boundary = np.linspace(0,w[1])
                    y_last_decision_boundary = (w[2]/w[1])*x_last_decision_boundary
                    
                w += y*x            
                print("x: " + str(x))            
                print("w: " + str(w))
                # x向量 
                x_vector = np.linspace(0,x[1])
                y_vector = (x[2]/x[1])*x_vector
                
                # Decision boundary 的方向向量
                x_decision_boundary = np.linspace(-0.5,7)
                y_decision_boundary = (-w[1]/w[2])*x_decision_boundary - (w[0]/w[2])
                
                # Decision boundary 的法向量
                x_decision_boundary_normal_vector = np.linspace(0,w[1])
                y_decision_boundary_normal_vector = (w[2]/w[1])*x_decision_boundary_normal_vector

In [ ]:
pla(train_feature)

替代方案

In [ ]:
def naive_pla(datas):
    # 初始化權重,全給0
    w = datas[0][0]
    iteration = 0
    while True:
        iteration += 1
        false_data = 0

        for data in datas:
            # input與權重做內積
            t = dot(w, data[0])
            if sign(data[1]) != sign(t):
                # error是實際上的label值
                error = data[1]
                false_data += 1
                w += error * data[0]
        print 'iter%d (%d / %d)' % (iteration, false_data, len(datas))
        if not false_data:
            break
    return w

In [ ]:
naive_pla()

### 2. PLA Pocket演算法

In [ ]:
def pocket_pla(datas, limit):
    
    
    def _calc_false(vec):
        res = 0
        for data in datas:
            t = np.dot(vec, data[0])
            if np.sign(data[1]) != np.sign(t):
                res += 1
        return res
    
    
    w = np.random.rand(5)
    least_false = _calc_false(w)
    res = w

    for i in xrange(limit):
        data = random.choice(datas)
        t = np.dot(w, data[0])
        if np.sign(data[1]) != np.sign(t):
            t = w + data[1] * data[0]
            t_false = _calc_false(t)

            w = t

            if t_false <= least_false:
                least_false = t_false
                res = t
    return res, least_false

In [ ]:
pocket_pla()

## 預測

In [70]:
def pla_predict(data):
    predict = []
    for ix in range(len(data.index)):
        inputVector = data.iloc[ix]
        global weights, bias
        dotproduct = np.dot(inputVector, weights)  # take the dot product of input and weight
        # bias 目前只會降低準確度
        # dotproduct = dotproduct + bias
        estimatedLabel = np.sign(dotproduct)
        if estimatedLabel == -1:
            estimatedLabel = 0
        else:
            estimatedLabel = 1
        predict.append(estimatedLabel) 
        
    return predict

In [71]:
test = pd.read_csv('test.csv')

test_feature = test.drop(['Name', 'Ticket', 'PassengerId','Cabin'], axis=1)
median_age = test_feature['Age'].dropna().median()
test_feature.loc[ (test_feature['Age'].isnull()), 'Age'] = median_age
test_feature['Embarked'] = test_feature['Embarked'].fillna(test_feature['Embarked'].mode()[0])
test_feature['Embarked'] = test_feature['Embarked'].map({'C': 0, 'Q': 1, 'S':2}).astype(int)
test_feature['Sex'] = test_feature['Sex'].map({'male': 0, 'female': 1}).astype(int)

# test['Title'] = test['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
# test['TitleGroup'] = test['Title'].map(title_dict)
# test_feature = pd.concat([test_feature, pd.get_dummies(test['TitleGroup'])], axis=1)
# test_feature.insert(loc=10, column='Noble', value=0)

# fare有空值
test_feature.loc[ (test_feature['Fare'].isnull()), 'Fare'] = test_feature['Fare'].dropna().median()
test_feature[['Pclass', 'Age', 'Parch', 'Fare']] = scaler.transform(test_feature[['Pclass', 'Age', 'Parch', 'Fare']])

y_pred = pla_predict(test_feature)

### 計算模型準確度

In [72]:
from sklearn.metrics import accuracy_score
answer = pd.read_csv('submission.csv')

accuracy_score(answer['Survived'], y_pred)

0.7607655502392344

#### 輸出成csv

In [73]:
DF_PLA = pd.DataFrame(
    {'PassengerId': answer['PassengerId'],
     'Survived': y_pred
    })

DF_PLA.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [74]:
DF_PLA.to_csv('submit_PLA.csv', index=False)